In [54]:
!pip install Bani

Found existing installation: Bani 0.6
Uninstalling Bani-0.6:
  Would remove:
    /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/Bani-0.6.dist-info/*
    /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/Bani/*
Proceed (y/n)? ^C
ERROR: Operation cancelled by user


In [ ]:
!python -m spacy download en_core_web_md

In [1]:
from Bani.Bani import Bani
from Bani.core.FAQ import FAQ

In [2]:
"""
This cell is optional, we read the data from any faq here
In this case we use the orignal faq pickle files , you may use
csv or any other format !!!


THE goal here is to get a list of questions and corosponding answers
"""
import pickle
def save_dict(obj, path):
    with open(path, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_dict(path):
    with open(path, 'rb') as f:
        return pickle.load(f)
    
def extractQA(orignalDct):
    """
    Just returns the questions and answers in a list , 
    NOthing fancy
    """
    q2L = orignalDct["question_to_label"]
    a2L = orignalDct["answer_to_label"]
    

    l2A = dict()
    for a,l in a2L.items():
        l2A[l] = a


    questions = []
    answers = []

    for q,l in q2L.items():
        questions.append(q)
        answers.append(l2A[l])
    return questions , answers
    
babyBonusOrignal = load_dict("./tutorialFAQs/baby_bonus_orignal.pkl") 
comcareOrignal =  load_dict("./tutorialFAQs/comcare_orignal.pkl")

b_questions , b_answers = extractQA(babyBonusOrignal)  # list of questions and answers 
c_questions , c_answers = extractQA(comcareOrignal)    # list of questions and answers 

In [38]:
"""
Now the real work starts , We have question answer pairs from 
baby bonus and comcare , We need to create a new faq for both of them

NOTE - This is the starting point , we need a list of questions and a list of answers
THE LENGTH OF BOTH LIST MUST BE SAME
if two questions answer the same questions then just copy the answer 
"""

# first we convert the questions and answers to FAQs
babyBonusFAQ = FAQ(name = "babyBonus",questions = b_questions, answers = b_answers)
comcareFAQ = FAQ(name = "comcare", questions = c_questions , answers = c_answers)

In [3]:
"""
Until this point we only have the orignal questions and answers in the faq
We now need to do two things
1) generate questions using pipeline
2) assign vectors to the questions using a model  


you can use your own generation pipeline , or use the default pipeline provided !!!
if you choose to implement your own pipeline make sure that your class implements at least one of 
batch_generate or exact_batch_generate methods


"""
from Bani.core.defaults import defaultGenerateManager


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [21]:
"""
This is an example of creating your own GenerateManager

"""

from Bani.core.generation import GenerateManager

class MyProducer1:
    def __init__(self):
        pass
    
    def batch_generate(questions : List[str]) -> Dict[str, List[str]]:
        """
        Takes list of questions and returns a dict , with each question 
        mapped to the list of generated questions
        """
        
        resultDict = dict()
        for question in questions:
            resultDict[question] = ["generated1", "generated2", "and so on"]
        
        return resultDict
    
class MyProducer2:
    def __init__(self):
        pass
    
    def generate(self, sentence):
        # Implement your generation of questions for a given sentence
        # Return a list
        pass
    
    def batch_generate(questions : List[str]) -> Dict[str, List[str]]:
        """
        Takes list of questions and returns a dict , with each question 
        mapped to the list of generated questions
        """
        
        resultDict = dict()
        for question in questions:
            resultDict[question] = self.generate(sentence)
        
        return resultDict
    
    
names = ["myProducer1_name", "myProducer2_name"]
toGenerate = [3,5] # At max generate 3 for first producer and 5 for second
producers = [MyProducer1(), MyProducer2()]


myGenerateManager = GenerateManager(producers = producers , names = names , nums = toGenerate)

NameError: name 'List' is not defined

In [39]:
"""
Now we will build the FAQ using the generationManager
This will generate paraphrases of the questions using the pipeline given

you can also leave the generator argument as none ,
In this case you wont be generating any questions, but then you cant train using a 
batch hard triple loss as well !!!

"""

# babyBonusFAQ.buildFAQ(generator = defaultGenerateManager)
babyBonusFAQ.buildFAQ(generator = None)

working with IdentityProducer pipeline


In [40]:
# comcareFAQ.buildFAQ(generator  = defaultGenerateManager)
comcareFAQ.buildFAQ(generator  = None)

working with IdentityProducer pipeline


In [10]:
"""
Now we can save the FAQ at any time we want !! , just pass the path to the root dir 
"""
babyBonusFAQ.save("./faqStore")
comcareFAQ.save("./faqStore")

In [2]:
"""
The saved FAQ can be easily loaded 
"""
babyBonusFAQ = FAQ("babyBonus")
babyBonusFAQ.load("./faqStore")

In [3]:
comcareFAQ = FAQ("comcare")
comcareFAQ.load("./faqStore")

In [4]:
"""
Lets now take a look at the bot part, for assignVectors refer to the readme
JR: If modelPath is None, download a pretrained model of the SentenceTransformer
"""
bot = Bani(modelPath = None , FAQs =[babyBonusFAQ, comcareFAQ], assignVectors = True)

OverWritting the vectors of FAQ named babyBonus , it will name save the generated vectors , to do that use the saveFAQ/s feature
OverWritting the vectors of FAQ named comcare , it will name save the generated vectors , to do that use the saveFAQ/s feature


In [8]:
"""
We can also save the FAQ again to preserve the assigned vectors ,
The next time you load these FAQs with the bot the vectors will not be reassigned ,
You need to clear the assigned vectors if you want to reassign
"""
bot.saveFAQs("./faqStore")


In [31]:
"""
out is a list of FAQ output , each output for each FAQ
compare the score field in each to get the highest scored FAQ, 
you can also set a margin of error and ask the user to decide between the faq

"""
out = bot.findClosest("What is the foreign domestic worker (FDW) grant?", K = 5)


In [24]:
out

In [25]:
print(out.faqId, out.faqName)

AttributeError: 'list' object has no attribute 'faqId'

In [17]:
"""
Finding answer from a specific FAQ
"""
out = bot.findClosestFromFAQ(1,"eligliblity for baby bonus for my child?", K = 3, topSimilar= 5)

In [30]:
print(out[0])

faqName - babyBonus

answer - If your spouse is a foreigner, when filling in the Baby Bonus application form, you should  Leave the “NRIC/FIN” field blank Select “Passport Number” or “Foreign ID” for “Identification Type”, and Enter his or her latest passport number at “Passport Number/Foreign ID” field.  

question - My spouse is a foreigner. How should I complete his/her details in the application form?

maxScore - 0.5470118522644043

score - 2.6833885312080383


['What other benefits can unwed parents qualify for?', 'How much does an organisation need to pay to register as an Approved Institution (AI) with Ministry of Social and Family Development (MSF)?', 'Why does my spouse need to log in with SingPass as well? Can I submit the application on my spouseâ€™s behalf?', 'Is there a limit to the number of Child Development Accounts per family?', 'How do I receive the Baby Bonus letters from Ministry of Social and Family Development (MSF) if I am overseas?']


In [8]:
"""
Training
"""

'\nTraining\n'

In [ ]:
"""
Look at the documentation for train , and set all the arguments accordingly,
if you have not run the buildFAQ , then you cannot train on most of the losses !!! 
losses available are batchHardTriplet and contrastiveLoss
"""
bot.train("./checkpoints/dummy", lossName = "softmaxLayerLos1s", batchSize = 8, warmst)

In [ ]:
"""
Testing
"""

In [5]:
import pandas as pd

In [6]:
df = pd.read_csv("./tutorialFAQs/babybonusTest.csv")

In [ ]:
df.columns

In [7]:
testData = []

for i in range(len(df)):
    orignal = df.loc[i,"original"]
    reframed = df.loc[i,"reframed"]
    
    testData.append((orignal,reframed))

In [8]:
"""
testing requires a list of (question , reframed_question) to be passed and returns the accuracy score 
you can test one faq at a time
"""
correct, question_count = bot.test(faqId = 0, testData = testData, K = 3)
print(f"{correct/question_count} with {correct} out of {question_count} questions correct")

/Users/james/Projects/Bani/Bani/Bani.py:204: UserWarning: 3 questions in the test set did not match any orignal questions
  warnings.warn("{} questions in the test set did not match any orignal questions".format(len(nonMatched)))


Running test on 291 questions
0.9140893470790378 with 266 out of 291 questions correct


## Testing Bani with different datasets (augmented or original FAQs) with or without training (James) to explore the effects of using augmented FAQs and performing model training for transfer learning

In [9]:
"""
Test with augmented FAQs (w/o training)
"""
from Bani.Bani import Bani
from Bani.core.FAQ import FAQ
import pandas as pd

# Load FAQs with augmented questions
babyBonusFAQ = FAQ("babyBonus")
babyBonusFAQ.load("./faqStore")
comcareFAQ = FAQ("comcare")
comcareFAQ.load("./faqStore")

# Initialize Bani
bot = Bani(modelPath = None , FAQs =[babyBonusFAQ, comcareFAQ], assignVectors = True)

# Do I need to run this?
# bot.train(outputPath = "./checkpoints/dummy", lossName = "batchHardTriplet", batchSize = 64)

# Prepare test data
df = pd.read_csv("./tutorialFAQs/babybonusTest.csv")
testData = []

for i in range(len(df)):
    orignal = df.loc[i,"original"]
    reframed = df.loc[i,"reframed"]
    
    testData.append((orignal,reframed))

# Begin test
correct, question_count = bot.test(faqId = 0, testData = testData, K = 3)
print(f"{correct/question_count} with {correct} out of {question_count} questions correct")

OverWritting the vectors of FAQ named babyBonus , it will name save the generated vectors , to do that use the saveFAQ/s feature
OverWritting the vectors of FAQ named comcare , it will name save the generated vectors , to do that use the saveFAQ/s feature
Running test on 291 questions
0.9140893470790378 with 266 out of 291 questions correct


In [10]:
"""
Test with augmented FAQs (with training via batchHardTriplet)
"""
from Bani.Bani import Bani
from Bani.core.FAQ import FAQ
import pandas as pd

# Load FAQs with augmented questions
babyBonusFAQ = FAQ("babyBonus")
babyBonusFAQ.load("./faqStore")
comcareFAQ = FAQ("comcare")
comcareFAQ.load("./faqStore")

# Initialize Bani
bot = Bani(modelPath = None , FAQs =[babyBonusFAQ, comcareFAQ], assignVectors = True)

# Do I need to run this?
bot.train(outputPath = "./checkpoints/dummy", lossName = "batchHardTriplet", batchSize = 64,epochs=1)

# Prepare test data
df = pd.read_csv("./tutorialFAQs/babybonusTest.csv")
testData = []

for i in range(len(df)):
    orignal = df.loc[i,"original"]
    reframed = df.loc[i,"reframed"]
    
    testData.append((orignal,reframed))

# Begin test
correct, question_count = bot.test(faqId = 0, testData = testData, K = 3)
print(f"{correct/question_count} with {correct} out of {question_count} questions correct")

OverWritting the vectors of FAQ named babyBonus , it will name save the generated vectors , to do that use the saveFAQ/s feature
OverWritting the vectors of FAQ named comcare , it will name save the generated vectors , to do that use the saveFAQ/s feature




Assigning vectors from the trained model to FAQ babyBonus
Assigning vectors from the trained model to FAQ comcare
Running test on 291 questions
0.9072164948453608 with 264 out of 291 questions correct


In [5]:
from Bani.core.defaults import defaultGenerateManager
babyBonusOrignal = load_dict("./tutorialFAQs/baby_bonus_orignal.pkl") 
comcareOrignal =  load_dict("./tutorialFAQs/comcare_orignal.pkl")

b_questions , b_answers = extractQA(babyBonusOrignal)  # list of questions and answers 
c_questions , c_answers = extractQA(comcareOrignal)    # list of questions and answers 
babyBonusFAQ = FAQ(name = "babyBonus",questions = b_questions, answers = b_answers)
comcareFAQ = FAQ(name = "comcare", questions = c_questions , answers = c_answers)

babyBonusFAQ.buildFAQ(generator = defaultGenerateManager)
comcareFAQ.buildFAQ(generator = defaultGenerateManager)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


working with SymSub pipeline
Initializing spaCy model...


  0%|          | 0/290 [00:00<?, ?it/s]

working with FPM pipeline


 20%|██        | 58/290 [00:00<00:00, 577.15it/s]

...

 21%|██        | 60/288 [00:00<00:00, 593.25it/s]

working with EDA pipeline


  0%|          | 0/288 [00:00<?, ?it/s]

working with nlpAug pipeline


100%|██████████| 288/288 [03:45<00:00,  1.28it/s]


In [ ]:
from Bani.Bani import Bani
from Bani.core.FAQ import FAQ

# Load FAQs with augmented questions
babyBonusFAQ = FAQ("babyBonus")
babyBonusFAQ.load("./jamesFAQs/Augmented")
comcareFAQ = FAQ("comcare")
comcareFAQ.load("./jamesFAQs/Augmented")

# Initialize Bani
bot = Bani(modelPath = None , FAQs =[babyBonusFAQ, comcareFAQ],assignVectors = True)

Assigning vectors to babyBonus faq , , it will name save the generated vectors , to do that use the saveFAQ/s feature


In [ ]:
"""
Regenerate questions + Test with augmented FAQs (with training via batchHardTriplet)
"""
from Bani.Bani import Bani
from Bani.core.FAQ import FAQ
from Bani.core.defaults import defaultGenerateManager
import pandas as pd

# babyBonusOrignal = load_dict("./tutorialFAQs/baby_bonus_orignal.pkl") 
# comcareOrignal =  load_dict("./tutorialFAQs/comcare_orignal.pkl")

# b_questions , b_answers = extractQA(babyBonusOrignal)  # list of questions and answers 
# c_questions , c_answers = extractQA(comcareOrignal)    # list of questions and answers 
# babyBonusFAQ = FAQ(name = "babyBonus",questions = b_questions, answers = b_answers)
# comcareFAQ = FAQ(name = "comcare", questions = c_questions , answers = c_answers)

# babyBonusFAQ.buildFAQ(generator = defaultGenerateManager)
# comcareFAQ.buildFAQ(generator = defaultGenerateManager)

# babyBonusFAQ.save("./jamesFAQs/Augmented")
# comcareFAQ.save("./jamesFAQs/Augmented")

# Load FAQs with augmented questions
# babyBonusFAQ = FAQ("babyBonus")
# babyBonusFAQ.load("./jamesFAQs/Augmented")
# comcareFAQ = FAQ("comcare")
# comcareFAQ.load("./jamesFAQs/Augmented")

# # Initialize Bani
# bot = Bani(modelPath = None , FAQs =[babyBonusFAQ, comcareFAQ])

# Do I need to run this?
new_model_path= "./checkpoints/latest_model1"
bot.train(outputPath = new_model_path, lossName = "batchHardTriplet", batchSize = 64,epochs=5)

# # Prepare test data
# df = pd.read_csv("./tutorialFAQs/babybonusTest.csv")
# testData = []

# for i in range(len(df)):
#     orignal = df.loc[i,"original"]
#     reframed = df.loc[i,"reframed"]
    
#     testData.append((orignal,reframed))

# # Begin test
# correct, question_count = bot.test(faqId = 0, testData = testData, K = 3)
# print(f"{correct/question_count} with {correct} out of {question_count} questions correct")

In [ ]:
# Prepare test data
df = pd.read_csv("./tutorialFAQs/babybonusTest.csv")
testData = []

for i in range(len(df)):
    orignal = df.loc[i,"original"]
    reframed = df.loc[i,"reframed"]
    
    testData.append((orignal,reframed))

# Begin test
correct, question_count = bot.test(faqId = 0, testData = testData, K = 3)
print(f"{correct/question_count} with {correct} out of {question_count} questions correct")

In [1]:
"""
Test with original FAQs (w/o training)
"""
from Bani.Bani import Bani
from Bani.core.FAQ import FAQ
import pandas as pd

# Load FAQs with augmented questions
babyBonusFAQ = FAQ("babyBonus")
babyBonusFAQ.load("./no_generate")
comcareFAQ = FAQ("comcare")
comcareFAQ.load("./no_generate")

# Initialize Bani
bot = Bani(modelPath = None , FAQs =[babyBonusFAQ, comcareFAQ], assignVectors = True)

# Do I need to run this?
# bot.train(outputPath = "./checkpoints/dummy", lossName = "batchHardTriplet", batchSize = 64)

# Prepare test data
df = pd.read_csv("./tutorialFAQs/babybonusTest.csv")
testData = []

for i in range(len(df)):
    orignal = df.loc[i,"original"]
    reframed = df.loc[i,"reframed"]
    
    testData.append((orignal,reframed))

# Begin test
correct, question_count = bot.test(faqId = 0, testData = testData, K = 3)
print(f"{correct/question_count} with {correct} out of {question_count} questions correct")

Assigning vectors to babyBonus faq , , it will name save the generated vectors , to do that use the saveFAQ/s feature
Assigning vectors to comcare faq , , it will name save the generated vectors , to do that use the saveFAQ/s feature


/Users/james/Projects/Bani/Bani/Bani.py:200: UserWarning: 4 questions in the test set did not match any orignal questions
  warnings.warn("{} questions in the test set did not match any orignal questions".format(len(nonMatched)))


Running test on 290 questions
0.903448275862069 with 262 out of 290 questions correct


In [14]:
"""
Test with original FAQs (with training via batchHardTriplet)
You cannot perform batchHardTriplet loss training without alternative examples of the same question
"""
from Bani.Bani import Bani
from Bani.core.FAQ import FAQ
import pandas as pd

# Load FAQs with augmented questions
babyBonusFAQ = FAQ("babyBonus")
babyBonusFAQ.load("./no_generate")
comcareFAQ = FAQ("comcare")
comcareFAQ.load("./no_generate")

# Initialize Bani
bot = Bani(modelPath = None , FAQs =[babyBonusFAQ, comcareFAQ], assignVectors = True)

# Do I need to run this?
bot.train(outputPath = "./checkpoints/dummy", lossName = "batchHardTriplet", batchSize = 64)

# Prepare test data
df = pd.read_csv("./tutorialFAQs/babybonusTest.csv")
testData = []

for i in range(len(df)):
    orignal = df.loc[i,"original"]
    reframed = df.loc[i,"reframed"]
    
    testData.append((orignal,reframed))

# Begin test
correct, question_count = bot.test(faqId = 0, testData = testData, K = 3)
print(f"{correct/question_count} with {correct} out of {question_count} questions correct")

Assigning vectors to babyBonus faq , , it will name save the generated vectors , to do that use the saveFAQ/s feature
Assigning vectors to comcare faq , , it will name save the generated vectors , to do that use the saveFAQ/s feature


TrainDataInvalid: Attempted to use batchHardTriplet loss but label 0 has only 1 exmaples !!! At least 2 required